# CER tests

Tests with character error rate (CER) computation with the goal of evaluating parts of documents that were generated with hand-written text recognition (HTR). For this purpose we use the [fastwer Python package](https://pypi.org/project/fastwer/) which can compute word error reates and character error rates for similar texts.

## 1. Initialization

In [ ]:
from collections import Counter
import fastwer
import os
import pandas as pd
import regex
import sys
sys.path.append(os.getcwd() + '/..')
from scripts import read_transkribus_files

In [ ]:
# validation data for HTR bestModel Curarao
data_dir_gold = "tmp/1586854/Validation_set/page"
data_dir_htr = "tmp/1749649/Validation_set_HTR_Curacao_bestModel/page"
# single page test of Loghi with the above validation data
data_dir_htr = "../../loghi/images.20240126.01/page.republicprint.baseline"
# sample test 1: 100 newly annotated files from January 2024
###data_dir_gold= "tmp/1616639/Sample_test_1/page"
###data_dir_htr = "tmp/1764847/Sample_test_1/page"
# ten-page test with loghi; documents from sample test 1
###data_dir_htr = "../../loghi/images/page"

Training data for model Dutchess Curacao (276):

* Training_extra_0001 - Training_extra_0011 (11)
* p001 - p100 (skipped p010, p020, *p021*, p030, *p036*, p040, *p044*, p050, p060, p070, p080, p090, *p098*, p100) (86)
* Sample_regex-0001 - Sample_regex-0100 (skipped: 10, 20, 30, 40, 50, 60, 70, 80, 90, 100) (90)
* p055 - p100 (skipped 55, 62, 71, 85, 90) 
* p001 - p054 (skipped 5, 12, 18, 31, 43, 51) - 54: 25-08-1880 577 (89)

Validation data (35):

- Validation_extra-0001 - Validation_extra-0005 (skipped 3) (4)
- p010 - p100 (10)
- Sample_regex-0010 - Sample_regex-0100 (10)
- p055, 62, 71, 85, 90
- p005, 12, 18, 31, 43, 51 (11)

## 2. Read files

In [ ]:
texts_gold, metadata_gold, textregions_gold  = read_transkribus_files.read_files(data_dir_gold)
texts_htr, metadata_htr, textregions_htr  = read_transkribus_files.read_files(data_dir_htr)

In [ ]:
if not regex.search("loghi", data_dir_htr):
    for file_name in texts_htr:
        print(f"{file_name}: combining {len(texts_htr[file_name])} text regions")
        texts_htr[file_name] = "".join(texts_htr[file_name])
else:
    if len(texts_htr) != 1 or list(texts_htr.keys())[0] != "p010.xml":
        print("unexpected loghi data: {texts_htr.keys()}")
    elif regex.search("cBAD", data_dir_htr):
        texts_htr["p010.xml"] = (texts_htr["p010.xml"][0] +
                                 texts_htr["p010.xml"][1] +
                                 texts_htr["p010.xml"][2] +
                                 texts_htr["p010.xml"][3] +
                                 texts_htr["p010.xml"][4] +
                                 texts_htr["p010.xml"][7] +
                                 texts_htr["p010.xml"][5] +
                                 texts_htr["p010.xml"][4])
    elif regex.search("general", data_dir_htr):
        texts_htr["p010.xml"] = (texts_htr["p010.xml"][0] +
                                 texts_htr["p010.xml"][6] +
                                 texts_htr["p010.xml"][1] +
                                 texts_htr["p010.xml"][2] +
                                 texts_htr["p010.xml"][3] +
                                 texts_htr["p010.xml"][5] +
                                 texts_htr["p010.xml"][4])
    elif regex.search("republic\.", data_dir_htr):
        texts_htr["p010.xml"] = (texts_htr["p010.xml"][0] +
                                 texts_htr["p010.xml"][10] +
                                 texts_htr["p010.xml"][1] +
                                 texts_htr["p010.xml"][2] +
                                 texts_htr["p010.xml"][9] +
                                 texts_htr["p010.xml"][3] +
                                 texts_htr["p010.xml"][4] +
                                 texts_htr["p010.xml"][5] +
                                 texts_htr["p010.xml"][8] +
                                 texts_htr["p010.xml"][6] +
                                 texts_htr["p010.xml"][7])
    elif regex.search("republicprint", data_dir_htr):
        texts_htr["p010.xml"] = (texts_htr["p010.xml"][0] +
                                 texts_htr["p010.xml"][32] +
                                 texts_htr["p010.xml"][27] +
                                 texts_htr["p010.xml"][1] +
                                 texts_htr["p010.xml"][23] +    
                                 texts_htr["p010.xml"][12] +
                                 texts_htr["p010.xml"][7] +
                                 texts_htr["p010.xml"][16] +
                                 texts_htr["p010.xml"][10] +
                                 texts_htr["p010.xml"][2] +
                                 texts_htr["p010.xml"][3] +
                                 texts_htr["p010.xml"][4] +
                                 texts_htr["p010.xml"][5] +
                                 texts_htr["p010.xml"][6] +
                                 texts_htr["p010.xml"][8] +
                                 texts_htr["p010.xml"][9] +
                                 texts_htr["p010.xml"][11] +
                                 texts_htr["p010.xml"][13] +
                                 texts_htr["p010.xml"][14] +
                                 texts_htr["p010.xml"][15] +
                                 texts_htr["p010.xml"][17] +
                                 texts_htr["p010.xml"][18] +
                                 texts_htr["p010.xml"][19] +
                                 texts_htr["p010.xml"][20] +
                                 texts_htr["p010.xml"][21] +
                                 texts_htr["p010.xml"][22] +
                                 texts_htr["p010.xml"][24] +
                                 texts_htr["p010.xml"][25] +
                                 texts_htr["p010.xml"][26] +
                                 texts_htr["p010.xml"][28] +
                                 texts_htr["p010.xml"][29] +
                                 texts_htr["p010.xml"][30] +
                                 texts_htr["p010.xml"][31] +
                                 texts_htr["p010.xml"][33] +
                                 texts_htr["p010.xml"][34] +
                                 texts_htr["p010.xml"][35] +
                                 texts_htr["p010.xml"][36] +
                                 texts_htr["p010.xml"][37] +
                                 texts_htr["p010.xml"][38] +
                                 texts_htr["p010.xml"][39] +
                                 texts_htr["p010.xml"][40] +
                                 texts_htr["p010.xml"][41])

In [ ]:
for file_name in texts_gold:
    print(f"{file_name}: combining {len(texts_gold[file_name])} text regions")
    texts_gold[file_name] = "".join(texts_gold[file_name])

## 2. Compute CER per text

In [ ]:
def compute_cer_per_text(texts_htr, texts_gold):
    total_cer = 0
    total_chars = 0
    for file_name in sorted(texts_htr.keys()):
        cer = fastwer.score_sent(texts_htr[file_name], texts_gold[file_name], char_level=True)
        max_text_length = max(len(texts_htr[file_name]), len(texts_gold[file_name]))
        total_chars += max_text_length
        total_cer += max_text_length * cer
        print(f"{cer:4.1f}  {file_name}")
    print(f"average cer: {total_cer/total_chars:.1f}")

In [ ]:
compute_cer_per_text(texts_htr, texts_gold)

In [ ]:
pd.DataFrame([{"dataset": "CBAD", "order": [0,1,2,3,6,7,5,4], "#order": 8, "cer": 59},
              {"dataset": "general", "order": [0,6,1,2,3,5,4], "#order": 7, "cer": 22},
              {"dataset": "republic", "order": [0,10,1,2,9,3,4,5,8,6,7], "#order": 11, "cer": 38},
              {"dataset": "republicprint", 
               "order": [0,32,27,1,23,12,7,16,10,2,3,4,5,6,8,9,11,13,14,15,17,18,19,20,21,24,25,26,28,29,30,31,32,33,34,35,36,37,38,39,40,41], 
               "#order": 42, "cer": 62}])

## 3. Correct printed text

In [ ]:
def guess_year(text_htr):
    years = []
    numbers = { "een": 1, "twee": 2, "drie": 3, "vier": 4, "vijf": 5, "zes": 6, "zeven": 7, "acht": 8, "negen": 9, "tien": 10, 
                "elf": 11, "twaalf": 12, "dertien": 13, "veertien": 14, "vijftien": 15, "zestien": 16, "zeventien": 17, "achttien": 18, "negentien": 19,
                "twintig": 20, "dertig": 30, "veertig": 40, "vijftig": 50, "zestig": 60, "zeventig": 70, "tachtig": 80, "negentig": 90 }
    for line in text_htr.split("\n"):
        tokens = line.lower().split()
        for tokens_index in range(0, len(tokens)):
            if tokens[tokens_index] == "honderd" and tokens_index > 0:
                if tokens[tokens_index - 1] in [ "zeven", "zeventien" ]:
                    year = 1700
                elif tokens[tokens_index - 1] in [ "acht", "achttien" ]:
                    year = 1800
                elif tokens[tokens_index - 1] in [ "negen", "negentien" ]:
                    year = 1900
                else:
                    year = 0
                    print(f"unexpected token before \"honderd\": {tokens[tokens_index - 1]}")
                if tokens_index < len(tokens) - 1 and tokens[tokens_index +  1] in numbers:
                    year += numbers[tokens[tokens_index + 1]]
                if tokens_index < len(tokens) - 2 and tokens[tokens_index +  1] == "en" and tokens[tokens_index +  2] in numbers:
                    year += numbers[tokens[tokens_index + 2]]
                if tokens_index < len(tokens) - 3 and tokens[tokens_index +  2] == "en" and tokens[tokens_index +  3] in numbers:
                    year += numbers[tokens[tokens_index + 3]]                
                years.append(year)
    return years

In [ ]:
for file_name in texts_htr:
    print(guess_year(texts_htr[file_name]))

## 4. Find incorrect words

In [ ]:
def find_best_line_match(lines_htr, lines_gold, index_htr, gold_index_used, alignments):
    best_index_gold = -1
    best_cer = 100
    line_htr = lines_htr[index_htr]
    index_htr_old = -1
    for index_gold in range(0, len(lines_gold)):
        cer = fastwer.score_sent(line_htr, lines_gold[index_gold], char_level=True)
        if cer < best_cer and ( index_gold not in gold_index_used.keys() or cer < gold_index_used[index_gold][0]):
            best_cer = cer
            best_index_gold = index_gold
            if index_gold in gold_index_used:
                index_htr_old = gold_index_used[index_gold][1]
            gold_index_used[index_gold] = [ best_cer, index_htr ]
            if index_htr_old >= 0:
                best_index_gold_old, best_cer_old, gold_index_used, alignments = find_best_line_match(lines_htr, 
                                                                                                      lines_gold, 
                                                                                                      index_htr_old,
                                                                                                      gold_index_used,
                                                                                                      alignments)
                alignments[index_htr_old] = [ best_index_gold_old, best_cer_old ]
    alignments[index_htr] = [ best_index_gold, best_cer ]
    return best_index_gold, best_cer, gold_index_used, alignments

In [ ]:
def align_lines(lines_htr, lines_gold):
    alignments = {}
    gold_index_used = {}
    for index_htr in range(0, len(lines_htr)):
        best_index_gold, best_cer, gold_index_used, alignments = find_best_line_match(lines_htr, lines_gold, index_htr, gold_index_used, alignments)
    alignments = convert_alignments(alignments)
    return alignments

In [ ]:
def cleanup_line(line):
    line = line.lower()
    line = regex.sub("y", "ij", line)
    line = line.translate(str.maketrans("çéëóü", "ceeou"))
    line = regex.sub('^[.,!?:;")(-]+', "", line)
    line = regex.sub('\s[.,!?:;")(-]+', " ", line)
    line = regex.sub('[.,!?:;")(-]+\s', " ", line)
    line = regex.sub('[.,!?:;")(-]+$', "", line)
    return(line)

In [ ]:
def cleanup_text(text):
    return "\n".join([cleanup_line(line) for line in  text.split("\n")])

In [ ]:
def count_distances(alignments):
    return Counter([ alignment[1] - alignment[0] for alignment in alignments])

In [ ]:
def check_alignments_order(alignments):
    last_gold_index = -1
    to_be_deleted = []
    distances = count_distances(alignments)
    for alignment_index in range(0, len(alignments)):
        if last_gold_index < alignments[alignment_index][1]:
            last_gold_index = alignments[alignment_index][1]
        elif alignments[alignment_index][2] > alignments[alignment_index - 1][2]:
            to_be_deleted.append(alignment_index)
        elif alignments[alignment_index][2] < alignments[alignment_index - 1][2]:
            to_be_deleted.append(alignment_index - 1)
            last_gold_index = alignments[alignment_index][1]
        elif (distances[alignments[alignment_index][1] - alignments[alignment_index][0]] <
              distances[alignments[alignment_index - 1][1] - alignments[alignment_index - 1][0]]):
            to_be_deleted.append(alignment_index)
        else:
            to_be_deleted.append(alignment_index - 1)
            last_gold_index = alignments[alignment_index][1]
    for to_be_deleted_value in list(reversed(to_be_deleted)):
        alignments.pop(to_be_deleted_value)
    to_be_added = []
    for alignment_index in range(1, len(alignments)):
        if ((alignments[alignment_index][0] - alignments[alignment_index - 1][0] == 
             alignments[alignment_index][1] - alignments[alignment_index - 1][1]) and
            alignments[alignment_index][0] - alignments[alignment_index - 1][0] != 1):
            for alignment_index_delta in range(1, alignments[alignment_index][0] - alignments[alignment_index - 1][0]):
                to_be_added.append((alignment_index, 
                                    alignments[alignment_index - 1][0] + alignment_index_delta,
                                    alignments[alignment_index - 1][1] + alignment_index_delta))
    for to_be_added_element in list(reversed(to_be_added)):
        alignments.insert(to_be_added_element[0], (to_be_added_element[1],
                                                   to_be_added_element[2],
                                                   fastwer.score_sent(lines_htr[to_be_added_element[1]], 
                                                                      lines_gold[to_be_added_element[2]], 
                                                                      char_level=True)))
    return len(to_be_deleted) > 0 or len(to_be_added) > 0, alignments

In [ ]:
def check_alignments_order_wrapper(alignments):
    alignments_changed = True
    while alignments_changed:
        alignments_changed, alignments = check_alignments_order(alignments)
    return alignments

In [ ]:
def fix_split_words(words_htr, words_gold, wrong_words, missed_words):
    to_be_deleted = []
    for index_wrong in range(1, len(wrong_words)):
        if wrong_words[index_wrong] == wrong_words[index_wrong - 1] + 1:
            combined_word = (words_htr[wrong_words[index_wrong - 1]] + 
                             words_htr[wrong_words[index_wrong]])
            for index_missed in range(0, len(missed_words)):
                if words_gold[missed_words[index_missed]] == combined_word:
                    to_be_deleted.append((index_wrong - 1, index_wrong, index_missed))
                    break
    for to_be_deleted_item in list(reversed(to_be_deleted)):
        for to_be_deleted_wrong_index in range(to_be_deleted_item[1], to_be_deleted_item[0] - 1, -1):
            wrong_words.pop(to_be_deleted_wrong_index)
        missed_words.pop(to_be_deleted_item[2])
    return wrong_words, missed_words

In [ ]:
def analyze_words(line_htr, line_gold):
    missed_words = []
    wrong_words = []
    if line_htr != line_gold:
        words_htr = line_htr.split()
        words_gold = line_gold.split()
        alignments = align_lines(words_htr, words_gold)
        alignments = check_alignments_order_wrapper(alignments)
        index_htr = 0
        index_gold = 0
        for index_alignment in range(0, len(alignments)):
            target_index_htr = alignments[index_alignment][0]
            target_index_gold = alignments[index_alignment][1]
            while index_htr < len(words_htr) and index_htr < target_index_htr:
                wrong_words.append(index_htr)
                index_htr += 1
            while index_gold < len(words_gold) and index_gold < target_index_gold:
                missed_words.append(index_gold)
                index_gold += 1
            if words_htr[target_index_htr] != words_gold[target_index_gold]:
                missed_words.append(target_index_gold)
                wrong_words.append(index_htr)
            index_htr += 1
            index_gold += 1
        for index_htr_extra in range(index_htr, len(words_htr)):
            wrong_words.append(index_htr)
        for index_gold_extra in range(index_gold, len(words_gold)):
            missed_words.append(index_gold_extra)
        wrong_words, missed_words = fix_split_words(words_htr, words_gold, wrong_words, missed_words)
    return wrong_words, missed_words

In [ ]:
def analyze_lines(lines_htr, lines_gold, alignments):
    index_htr = 0
    line_analysis = []
    for alignment in alignments:
        for index_htr_delta in range(1, alignment[0]-index_htr):
             line_analysis.append(analyze_words(lines_htr[index_htr + index_htr_delta], ""))
        line_analysis.append(analyze_words(lines_htr[alignment[0]], lines_gold[alignment[1]]))
        index_htr = alignment[0] + 1
    for index_htr_delta in range(1, len(lines_htr)-index_htr):
        line_analysis.append(analyze_words(lines_htr[index_htr + index_htr_delta], ""))
    return line_analysis

In [ ]:
def show_word_analysis(line_htr, line_gold, line_analysis_line):
    words_htr = line_htr.split()
    words_gold = line_gold.split()
    for index_htr in range(0, len(words_htr)):
        if index_htr in line_analysis_line[0]:
            read_transkribus_files.print_with_color(words_htr[index_htr], color_code=1, end=" ")
        else:
            read_transkribus_files.print_with_color(words_htr[index_htr], color_code=0, end=" ")
    if len(line_analysis_line[1]) > 0:
        read_transkribus_files.print_with_color([ words_gold[index_gold] 
                                                  for index_gold in line_analysis_line[1] ], 
                                                color_code=4, 
                                                end=" ")
    print()

In [ ]:
def show_line_analysis(lines_htr, lines_gold, alignments, line_analysis):
    index_htr = 0
    for alignment in alignments:
        for index_htr_extra in range(index_htr, alignment[0]):
            show_word_analysis(lines_htr[index_htr_extra], "", [[], []])
        show_word_analysis(lines_htr[alignment[0]], 
                           lines_gold[alignment[1]], 
                           line_analysis[alignment[0]])
        index_htr = alignment[0] + 1
    for index_htr_extra in range(index_htr, len(lines_htr)):
        show_word_analysis(lines_htr[index_htr_extra], "", [[], []])

In [ ]:
def convert_alignments(alignments):
    alignments_converted = []
    for alignments_key in alignments.keys():
        alignments_converted.append((alignments_key, alignments[alignments_key][0], alignments[alignments_key][1]))
    return alignments_converted

In [ ]:
for file_name in sorted(texts_htr.keys()):
    lines_htr = cleanup_text(texts_htr[file_name]).split("\n")
    lines_gold = cleanup_text(texts_gold[file_name]).split("\n")
    alignments = align_lines(lines_htr, lines_gold)
    cer = fastwer.score_sent(texts_htr[file_name], texts_gold[file_name], char_level=True)
    read_transkribus_files.print_with_color(f"{file_name} (cer={cer:.1f}):\n", color_code=4)
    line_analysis = analyze_lines(lines_htr, lines_gold, alignments)
    show_line_analysis(lines_htr, lines_gold, alignments, line_analysis)